## Instalación de OpenMC

In [0]:
#
# Executing this cell you will install OpenMC and the nuclear
# data libraries in this instance of the Google Colaboratory virtual machine.
# The process takes about 2 minutes.
#

def install_openmc():
  #
  # Clone source code from Github, make and install
  #
  
  import os
  
  if not os.path.isdir('/content'):
    print("Esta función instala OpenMC en una instancia de Google Colaboratory.")
    print("Para instalar localmente siga las instrucciones de la documentacion:")
    print("http://docs.openmc.org/en/stable/quickinstall.html")
    return
  
  %cd -q /content
  print("Obtaining OpenMC source code from Github...")
  !git  --no-pager clone https://github.com/mit-crpg/openmc.git &> /dev/null
  %cd -q openmc
  !git --no-pager checkout master &> /dev/null
  !mkdir build
  %cd -q build
  print("Running cmake...")
  !cmake ../ -DPYTHON_EXECUTABLE=/usr/bin/python3 -DCMAKE_INSTALL_PREFIX=/usr/local &> /dev/null
  print("Running make...")
  !make -j &> /dev/null
  print("Running make install...")
  !make install &> /dev/null
  import sys
  sys.path.append('/usr/local/lib/python3.6/dist-packages/openmc-0.10.0-py3.6-linux-x86_64.egg')
  sys.path.append('/usr/local/lib/python3.6/site-packages/')
  
  !apt update &> /dev/null
  !apt install python3-vtk7 imagemagick &> /dev/null
  
  %cd -q /content

def install_data_from_onedrive():
  #
  # Download preprocessed HDF5 files from OneDrive (faster).
  #
  import os
  
  if not os.path.isdir('/content'):
    print("Esta función instala los datos nucleares de OpenMC en una instancia de Google Colaboratory.")
    print("Para instalar localmente siga las instrucciones de la documentacion:")
    print("http://docs.openmc.org/en/stable/quickinstall.html")
    return
  %cd -q /content
  print("Obtaining HDF5 files from OneDrive...")
  !wget -O nndc_hdf5.tar.gz "https://onedrive.live.com/download?cid=22422A8EEA2A85B9&resid=22422A8EEA2A85B9%21105&authkey=AHL6xwYFXDwEzkk" &> /dev/null
  print("Uncompressing...")
  !tar xzvf nndc_hdf5.tar.gz &> /dev/null
    
from time import time
t1 = time()
install_openmc()
install_data_from_onedrive()
t2 = time()
print("Installed OpenMC in {:.2f} minutes".format((t2-t1)/60.0))


# Estructuras repetidas

In [0]:
import openmc
import numpy as np
%pylab inline

Vamos a definir un esquema de cuatro celdas en el plano $(x,y)$, una conteniendo una barra de combustible hexagonal, otra una barra de combustible de sección cuadrada y otra de sección cilíndrica.

In [0]:
agua = openmc.Material()
agua.add_nuclide("H1", 2.0, "ao")
agua.add_nuclide("O16", 1.0, "ao")
agua.set_density("g/cm3", 1.0)
fuel = openmc.Material()
fuel.add_nuclide("U235", 2.0, "ao")
fuel.add_nuclide("O16", 1.0, "ao")
fuel.set_density("g/cm3", 10.0)

Esta es la definición de una barra de sección hexagonal (infinita en $z$) con apotema $5.0$ cm.

Aquí vamos a usar la ecuación del plano de normal $\vec{n}$ que pasa por el punto $\vec{p}$:

$$
n_x(x-p_x)+n_y(y-p_y)+n_z(z-p_z)=0 
$$

$$
n_x x + n_y y + n_z z = n_x p_x+n_y p_y+n_z p_z
$$

Por lo que los parámetros $A$, $B$, $C$ y$D$ de la [definición del plano en OpenMC](https://openmc.readthedocs.io/en/stable/pythonapi/generated/openmc.Plane.html#openmc.Plane) son:

$$
A = n_x p_x
$$

$$
B = n_y p_y
$$

$$
C = n_z p_z
$$

$$
D = n_x p_x+n_y p_y+n_z p_z
$$

In [0]:
apotema = 5.0 
lado = apotema*2.0/np.sqrt(3)

ang = 30.0*np.pi/180.0
n = np.array([np.cos(ang), np.sin(ang), 0])
x0 = np.array([lado,0,0])
plano1 = openmc.Plane(A=n[0],B=n[1],C=n[2],D=np.dot(n, x0))

ang = 90.0*np.pi/180.0
n = np.array([np.cos(ang), np.sin(ang), 0])
x0 = np.array([0,apotema,0])
plano2 = openmc.Plane(A=n[0],B=n[1],C=n[2],D=np.dot(n, x0))

ang = 150.0*np.pi/180.0
n = np.array([np.cos(ang), np.sin(ang), 0])
x0 = np.array([-lado,0,0])
plano3 = openmc.Plane(A=n[0],B=n[1],C=n[2],D=np.dot(n, x0))

ang = 210.0*np.pi/180.0
n = np.array([np.cos(ang), np.sin(ang), 0])
x0 = np.array([-lado,0,0])
plano4 = openmc.Plane(A=n[0],B=n[1],C=n[2],D=np.dot(n, x0))

ang = 270.0*np.pi/180.0
n = np.array([np.cos(ang), np.sin(ang), 0])
x0 = np.array([0,-apotema,0])
plano5 = openmc.Plane(A=n[0],B=n[1],C=n[2],D=np.dot(n, x0))

ang = 330.0*np.pi/180.0
n = np.array([np.cos(ang), np.sin(ang), 0])
x0 = np.array([+lado,0,0])
plano6 = openmc.Plane(A=n[0],B=n[1],C=n[2],D=np.dot(n, x0))

hexa  = openmc.Cell(region=-plano1 & -plano2 & \
                           -plano3 & -plano4 & \
                           -plano5 & -plano6, fill=fuel)
fuera_hexa = openmc.Cell(region=+plano1 | +plano2 | \
                                +plano3 | +plano4 | \
                                +plano5 | +plano6, fill=agua)
uni1 = openmc.Universe(cells=[hexa,fuera_hexa])

Al utilizar el método `.plot()` para graficar un universo podemos especificar cómo se van a asignar los colores con el parámetro `color_by=` (el default es `"cell"`, pero vamos a colorear por material). La asignación de colores a materiales se realiza con el parámetro `colors=`, que recibe un diccionario cuyos índices son materiales:

In [0]:
uni1.plot(width=(20,20), color_by='material', colors={fuel:"Gray", agua:"Blue"})

La definición de la celda hexagonal puede hacerse en forma más sencilla usando la función [openmc.model.get_hexagonal_prism()](https://openmc.readthedocs.io/en/stable/pythonapi/generated/openmc.model.get_hexagonal_prism.html#openmc.model.get_hexagonal_prism) que devuelve directamente el interior de un prisma hexagonal infinito:

In [0]:
apotema = 5.0 
lado = apotema*2.0/np.sqrt(3)

region = openmc.model.get_hexagonal_prism(edge_length=lado, orientation='x')
hexa  = openmc.Cell(region=region, fill=fuel)
fuera_hexa = openmc.Cell(region=~region, fill=agua)
uni1 = openmc.Universe(cells=[hexa,fuera_hexa])
uni1.plot(width=(20,20), color_by='material', colors={fuel:"Gray", agua:"Blue"})

Esta es la definción de una barra de sección cuadrada (infinita en $z$):

In [0]:
plano11 = openmc.XPlane(x0 = -5)
plano12 = openmc.XPlane(x0 = +5)
plano13 = openmc.YPlane(y0 = -5)
plano14 = openmc.YPlane(y0 = +5)
box       = openmc.Cell(region=+plano11 & -plano12 & +plano13 & -plano14, fill=fuel)
fuera_box = openmc.Cell(region=-plano11 | +plano12 | -plano13 | +plano14, fill=agua)
uni2 = openmc.Universe(cells=[box, fuera_box])
uni2.plot(width=(20,20), color_by='material', colors={fuel:"Gray", agua:"Blue"})

La definición de la celda de sección rectangular puede hacerse en forma más sencilla usando la función [openmc.model.get_rectangular_prism()](https://openmc.readthedocs.io/en/stable/pythonapi/generated/openmc.model.get_rectangular_prism.html) que devuelve directamente el interior de un prisma rectangular infinito:

In [0]:
lado = 10
region = openmc.model.get_rectangular_prism(lado, lado)
box  = openmc.Cell(region=region, fill=fuel)
fuera_box = openmc.Cell(region=~region, fill=agua)
uni2 = openmc.Universe(cells=[box, fuera_box])

In [0]:
uni2.plot(width=(20,20), color_by='material', colors={fuel:"Gray", agua:"Blue"})

Y la definición de una barra de sección cilíndrica

In [0]:
zcil = openmc.ZCylinder(R=5)
cil       = openmc.Cell(region=-zcil, fill=fuel)
fuera_cil = openmc.Cell(region=+zcil, fill=agua)
uni3 = openmc.Universe(cells=[cil, fuera_cil])
uni3.plot(width=(20,20), color_by='material', colors={fuel:"Gray", agua:"Blue"})

Ahora vamos a definir las cuatro celdas que vamos a llenar con los universos predefinidos. Esto se hace asignando al parámetro `fill=` un universo en lugar de un material. Como los universos están definidos en el origen, debemos aplicar una traslación con para correr el universo a la posición final. Esto se hace asignando un valor al atributo `.translation`:

In [0]:
plano21 = openmc.XPlane(x0 = 0)
plano22 = openmc.XPlane(x0 = 15)
plano23 = openmc.XPlane(x0 = 30)
plano31 = openmc.YPlane(y0 = 0)
plano32 = openmc.YPlane(y0 = 15)
plano33 = openmc.YPlane(y0 = 30)

cel1 = openmc.Cell(region=+plano21 & -plano22 & +plano31 & -plano32, fill=uni1)
cel1.translation = [+7.5,+7.5,0]

cel2 = openmc.Cell(region=+plano22 & -plano23 & +plano31 & -plano32, fill=uni2)
cel2.translation = [+22.5,+7.5,0]

cel3 = openmc.Cell(region=+plano21 & -plano22 & +plano32 & -plano33, fill=uni3)
cel3.translation = [+7.5,+22.5,0]

cel4 = openmc.Cell(region=+plano22 & -plano23 & +plano32 & -plano33, fill=agua)

uni4 = openmc.Universe(cells=[cel1, cel2, cel3, cel4])

In [0]:
uni4.plot(width=(50,50), origin=(15,15,0), color_by='material', colors={fuel:"Gray", agua:"Blue"})

Si bien esto se puede hacer en forma repetida utilizando loops de Python, OpenMC posee también estructuras repetidas: los arreglos regulares denominados *lattices*. OpenMC posee dos tipos de lattices: rectangulares, que se definen con la función `RectLattice()`, y hexagonales que se definen con la función `HexLattice()`. Los lattices definen un arreglo matricial que se llena con universos, por lo que para definir una celda homogénea necesitamos definir un universo homogéneo:

In [0]:
cel_agua = openmc.Cell(fill=agua)
uni_agua = openmc.Universe(cells=[cel_agua])

(En otros códigos Monte Carlo, como MCNP, los lattices son tipos de celdas y poseen un material default que se asigna a las celdas vacías. En OpenMC los lattices existen en forma independiente de las celdas, por lo que no puede asignarseles materiales).

La definición de un lattice rectangular requiere la definición de la esquina inferior izquierda (con el atributo `.lower_left`) y el paso o *pitch* del arreglo (con el atributo `.pitch`). La asignación de universos se hace colocando en el atributo `.universes` una lista de listas conteniendo los universos que llenan el lattice:

In [0]:
lat1 = openmc.RectLattice()
lat1.lower_left = (0.0,0.0, 0.0)
lat1.pitch = (15.0, 15.0)
lat1.universes = [[uni3, uni_agua],\
                  [uni1, uni2]]

(Para definir un lattice en 3D se hace lo mismo, pero se asigna una tercera dimension al pitch, que será el paso en $z$. La definición de universos se hace con una lista triplemente anidada, donde cada lista `[[],[]]` corresponde a un nivel en $z$)

In [0]:
cel_lat1 = openmc.Cell(region=+plano21 & -plano23 & +plano31 & -plano33, fill=lat1)
uni5 = openmc.Universe(cells=[cel_lat1])
uni5.plot(width=(50,50), origin=(15,15,0), color_by='material', colors={fuel:"Gray", agua:"Blue"})

Como el ejemplo es sencillo no se aprecia la simplificación. Pero, la ventaja de los lattices es más clara cuando se quieren definir arreglos de dimensiones mayores. Por ejemplo, para definir un arreglo de combustibles de $9 \times 9$:

In [0]:
lat2 = openmc.RectLattice()
lat2.lower_left = (0.0,0.0, 0.0)
lat2.pitch = (20.0, 20.0)
lat2.universes = [[uni3, uni3, uni3, uni_agua, uni_agua, uni_agua, uni3, uni3, uni3],\
                  [uni3, uni3, uni3, uni_agua, uni_agua, uni_agua, uni3, uni3, uni3],\
                  [uni3, uni3, uni3, uni_agua, uni_agua, uni_agua, uni3, uni3, uni3],\
                  [uni1, uni1, uni1, uni2,     uni2,     uni2,     uni1, uni1, uni1],\
                  [uni1, uni1, uni1, uni2,     uni2,     uni2,     uni1, uni1, uni1],\
                  [uni1, uni1, uni1, uni2,     uni2,     uni2,     uni1, uni1, uni1],\
                  [uni3, uni3, uni3, uni_agua, uni_agua, uni_agua, uni3, uni3, uni3],\
                  [uni3, uni3, uni3, uni_agua, uni_agua, uni_agua, uni3, uni3, uni3],\
                  [uni3, uni3, uni3, uni_agua, uni_agua, uni_agua, uni3, uni3, uni3]]


In [0]:
plano24 = openmc.XPlane(x0 =9*20)
plano34 = openmc.YPlane(y0 =9*20)
cel_lat2 = openmc.Cell(region=+plano21 & -plano24 & +plano31 & -plano34, fill=lat2)
uni6 = openmc.Universe(cells=[cel_lat2])
uni6.plot(width=(10*20,10*20), origin=(4.5*20,4.5*20,0), \
          color_by='material', colors={fuel:"Gray", agua:"Blue"})

Los arreglos hexagonales son similares, pero el pitch define una distancia entre anillos concéntricos de combustible (nótese que el pitch es una tupla de un valor, no un escalar). Si se asigna un segundo valor al pitch será la separación entre planos en $z$. El llenado de los índices se realiza de la corona exterior a la corona interior. Para ayudar con el proceso, los lattices hexagonales poseen el método `.show_indices()` que devuelve las posiciones de los índices para una determinada cantidad de anillos.

In [0]:
lat3 = openmc.HexLattice()
lat3.center=(4.5*15, 4.5*15)
lat3.pitch=(20,)
print(lat3.show_indices(3))
lat3.universes = [[uni1, uni1, uni1, uni1, uni1, uni1, uni1, uni1, uni1, uni1, uni1, uni1],\
                  [uni3, uni3, uni3, uni3, uni3, uni3],\
                  [uni2]]
cel_lat3 = openmc.Cell(region=+plano21 & -plano24 & +plano31 & -plano34, fill=lat3)
uni7 = openmc.Universe(cells=[cel_lat3])
uni7.plot(width=(10*15,10*15), origin=(4.5*15,4.5*15,0), \
          color_by='material', colors={fuel:"Gray", agua:"Blue"})

Ahora vamos a hacer un modelo completo en base al lattice hexagonal. El primer paso va a ser rodear de agua los elementos combustibles para poder después utilizar esto como relleno para una celda cilíndrica de radio $R=60$ cm:

In [0]:
lat4 = openmc.HexLattice()
lat4.center=(0, 0)
lat4.pitch=(20,)
print(lat4.show_indices(5))

In [0]:
cil41 = openmc.ZCylinder(R=60, boundary_type="vacuum")
plano51 = openmc.ZPlane(z0=-60, boundary_type="vacuum")
plano52 = openmc.ZPlane(z0=+60, boundary_type="vacuum")
lat4.universes = [[uni_agua]*24,\
                  [uni_agua]*18,\
                  [uni1]*12,\
                  [uni3]*6,\
                  [uni2]]
cel_lat4 = openmc.Cell(region=-cil41&+plano51&-plano52, fill=lat4)
uni8 = openmc.Universe(cells=[cel_lat4])
uni8.plot(width=(120,120), \
          color_by='material', colors={fuel:"Gray", agua:"Blue"})
uni8.plot(width=(120,120))

(Una forma más sencilla de hacer lo mismo sería definir el universo default que llena las posiciones no especificadas del lattice. Esto se hace con el atributo `.outer` del lattice.)

Para verificar la geometría vamos a realizar un plot con el graficador del código (no el de Python). Para esto hay que crear un objeto tipo `Plot()` que luego se exportará al archivo `plots.xml`. Entre los parámetros a indicar hay que colocar el origen y dimensiones del gráfico y también la cantidad de píxeles en cada dimension:

In [0]:
geom = openmc.Geometry(root_universe=uni8)
geom.export_to_xml()
mats = openmc.Materials([agua, fuel])
mats.cross_sections="/content/nndc_hdf5/cross_sections.xml"
mats.export_to_xml()

In [0]:
plot2D = openmc.Plot()
plot2D.origin = (0, 0, 0)
plot2D.width = (150., 150.)
plot2D.pixels = (600, 600)
plot2D.id = 1
plot2D.filename = "grafico"
plots = openmc.Plots([plot2D])
plots.export_to_xml()
openmc.plot_geometry()

OpenMC genera un gráfico en formato ppm, que puede convertirse a otros formatos con el comando de Linux `convert`:

In [0]:
!convert grafico.ppm grafico.png

y graficarse en el notebook:

In [0]:
img = imread("grafico.png")
imshow(img)

Estos gráficos también pueden realizarse con el método `openmc.plot_inline()`:

In [0]:
openmc.plot_inline(plot2D)

El mismo método puede utilizarse para realizar gráficos 3D. En 3D el gráfico estará compuesto de *voxeles* (pixeles tridimensionales). Si el modelo es homogéneo en una dirección puede ponerse un sólo voxel para utilizar menos memoria:

In [0]:
plot3D = openmc.Plot()
plot3D.type = 'voxel'
plot3D.origin = (0, 0, 0)
plot3D.width = (150., 150., 150.)
plot3D.pixels = (600, 600, 1)
plot3D.id = 2
plot3D.filename = "grafico3D"
plots = openmc.Plots([plot3D])
plots.export_to_xml()
openmc.plot_geometry()

El archivo resultante (un archivo binario en formato HDF5) puede convertirse a format VTK para graficar con Paraview:

In [0]:
!openmc-voxel-to-silovtk -o grafico3D grafico3D.h5 

El proceso para visualizar en Paraview es:

* Abrir archivo `grafico3D.vti`.
* Cambiar la visibilidad haciendo click en el ojo al lado de `grafico3D.vti` en el *Pipeline browser* .
* En la barra de herramientas *Active Variable Controls* cambiar "Solid Colors" por "id" y "Outline" por "Surface".
* En *Properties* presionar "Apply".
* Conn el menú "Filters -> Common -> Clip" pueden realizarse cortes de la geometría en los distintos ejes para visualizar el interior.

In [0]:
from google.colab import files
files.download("grafico3D.vti")